In [1]:
from kickstart_knn import load_data

train = load_data("mlm_items_train.jsonlines")
test = load_data("mlm_items_test.jsonlines")

In [2]:
print train[0].keys()
print test[0].keys()

[u'seller_id', u'top_level_category', u'title', u'price', u'site', u'buying_mode', u'seller_power_status', u'sold_quantity', u'category_id', u'id', u'category_path']
[u'seller_id', u'title', u'price', u'site', u'buying_mode', u'seller_power_status', u'sold_quantity', u'id']


In [3]:
print train[0].values()

[75259727, u'MLM1648', u'Ventilador De Aspas Eva Usb Para Laptop Y Pc Flexible', 39, u'MLM', u'buy_it_now', u'platinum', 21, u'MLM54184', u'MLM505628392', [u'MLM1648', u'MLM1712', u'MLM10345', u'MLM54184']]


In [4]:
from random import shuffle

shuffle(train)
print train[0].values()

[141029774, u'MLM1574', u'Panel Solar 145w Kit Completo 12v Sistema Aislado A Cfe', 7950, u'MLM', u'buy_it_now', u'silver', 9, u'MLM1590', u'MLM507107571', [u'MLM1574', u'MLM1582', u'MLM1590']]


In [5]:
train_len = int(len(train)*0.9)
train_split = train[:train_len]
test_split = train[train_len:]

print len(train_split)
print len(test_split)

90000
10000


In [15]:
import pandas as pd
import numpy as np

y_train = pd.DataFrame(train_split)['top_level_category'].tolist()
columns = [u'seller_id', u'price', u'sold_quantity']

X_train = pd.DataFrame(train_split).as_matrix(columns).astype(np.float)

In [11]:
X_test = pd.DataFrame(test_split).as_matrix(columns)
y_test = pd.DataFrame(test_split)['top_level_category'].tolist()

Entrenamos el clasificador elegido.

In [12]:
#from sklearn.tree import DecisionTreeClassifier
#classifier = DecisionTreeClassifier()

from sklearn.linear_model import LinearRegression
classifier = LinearRegression(n_jobs=2)

In [13]:
classifier.fit(X_train, y_train)

TypeError: cannot perform reduce with flexible type

In [20]:
prediction = classifier.predict(X_test)
print prediction[0]
print y_test[0]

MLM1430
MLM1430


In [21]:
def calc_score(y_test, prediction):
    sames = sum(1 for y1, y2 in zip(y_test, prediction) if y1 == y2)
    return sames*1./len(y_test)

In [22]:
print calc_score(y_test, prediction)

0.4766


Vamos a probar cross val score, para variar el slice de test. KFold cross val. Por ejemplo si cortamos en slices de 10% para validar, es 10-fold cv.

In [24]:
from sklearn.cross_validation import cross_val_score

scores = cross_val_score(classifier, X_train, y=y_train, cv=10, n_jobs=2)
print scores.mean()

0.455790653357
